# Neural Prophet

Zbiór danych: https://www.kaggle.com/datasets/hamidrezabakhtaki/fmcg-export-sale

Dokumentacja: https://neuralprophet.com/code/forecaster.html

In [ ]:
#pip install neuralprophet 

In [ ]:
# Import pakietów
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from neuralprophet import NeuralProphet

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
# Przygotowanie danych panelowych
df_2 = pd.read_csv("data/ExportSales.csv")
df_2 = df_2[df_2['Product Code'].isin([30001278,
30001279,
30001280,
30001281,
30001282,
30001284,
30001285,
30001286,
30001287
])]
df_2['Date'] = pd.to_datetime(df_2['Date'])
df_2 = df_2.set_index('Date')
df_panel = df_2[['Product Code','QTY','Total Price']].groupby('Product Code')[['QTY','Total Price']].resample('ME').sum().reset_index()
df_panel['price'] = df_panel['Total Price'] / df_panel['QTY']
df_panel = df_panel[df_panel['Date']<='2022-07-31']
df_panel = df_panel.fillna(0)

In [3]:
# Dopasowanie kolumne do nazw w neuralprophet
df_panel = df_panel.rename(columns={'Date': 'ds' , 'QTY':'y' , 'Product Code':'ID'})

In [4]:
# train/test split
train =  df_panel[df_panel['ds']<='2022-04-30'].reset_index(drop=True)
test =  df_panel[df_panel['ds']>'2022-04-30'].reset_index(drop=True)

In [5]:
#  model
model = NeuralProphet()

In [ ]:
# Dodanie zmiennej egzogenicznej - każda zmienna musi być dodana osobnym wywołaniem metody
model.add_future_regressor('price')

In [ ]:
train.head()

In [8]:
# Usunięcie zbędnych zmiennych
del train['Total Price']
del test['Total Price']

In [ ]:
# fit
model.fit(train,freq= 'ME')

In [ ]:
# Prognoza
forecast = model.predict(df_panel[['ID','ds','y','price']])

In [ ]:
# sprawdzenie wyników
forecast

In [ ]:
# Ocena wizualna
for i in forecast['ID'].unique():
    df_to_plot = forecast[forecast['ID']==i]
    plt.plot(df_to_plot['ds'], df_to_plot['y'],label='actual')
    plt.plot(df_to_plot['ds'],df_to_plot['yhat1'],label='forecast')
    plt.title(f'ID: {i}')
    plt.legend()
    plt.show()